# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [ ]:
import requests
import time
import re
from functools import wraps

from random import randint

In [ ]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [ ]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)


    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end_time - start_time:.6f} секунд')
        return result
    return wrapper

In [ ]:
@benchmark
# def get_webpage():
#     import requests
#     webpage = requests.get('https://stepik.org')

# get_webpage()

Время выполнения функции get_webpage: 0.664468 секунд


In [ ]:
@benchmark
# def fib(n):
#     if n < 2:
#         return n
#     return fib(n-2) + fib(n-1)

# fib(3)

Функции fib потребовалось 0.000001 секунд
Функции fib потребовалось 0.000001 секунд
Функции fib потребовалось 0.000001 секунд
Функции fib потребовалось 0.001665 секунд
Функции fib потребовалось 0.003773 секунд


2

## Задание 2

In [ ]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @wraps(func)

    def wrapper(*args, **kwargs):
        args_str = ", ".join(repr(arg) for arg in args)
        kwargs_dict = {key: value for key, value in kwargs.items()}
        result = func(*args, **kwargs)
        print(f'Функция {func.__name__} вызвана с параметрами: ({args_str}), {kwargs_dict}')
        return result

    return wrapper

In [ ]:
@logging
# def final_price(*prices, discount=1, **kwargs):
#     low = kwargs.get("price_low", min(prices))
#     high = kwargs.get("price_high", max(prices))
#     return [price - price * discount / 100 for price in prices if low <= price <= high]

# print(final_price(100, 200, 300, 400, 500, discount=5))
# print(final_price(100, 200, 300, 400, 500, discount=5, price_low=200))
# print(final_price(100, 200, 300, 400, 500, discount=5, price_high=200))
# print(final_price(100, 200, 300, 400, 500, discount=5, price_low=200, price_high=350))

Функция final_price вызвана с параметрами: (100, 200, 300, 400, 500), {'discount': 5}
[95.0, 190.0, 285.0, 380.0, 475.0]
Функция final_price вызвана с параметрами: (100, 200, 300, 400, 500), {'discount': 5, 'price_low': 200}
[190.0, 285.0, 380.0, 475.0]
Функция final_price вызвана с параметрами: (100, 200, 300, 400, 500), {'discount': 5, 'price_high': 200}
[95.0, 190.0]
Функция final_price вызвана с параметрами: (100, 200, 300, 400, 500), {'discount': 5, 'price_low': 200, 'price_high': 350}
[190.0, 285.0]


## Задание 3

In [ ]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    @wraps(func)

    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        result = func(*args, **kwargs)
        print(f"Функция {func.__name__} была вызвана {wrapper.calls} раз(а)")
        return result

    wrapper.calls = 0
    return wrapper

In [ ]:
@counter
# def final_price(*prices, discount=1, **kwargs):
#     low = kwargs.get("price_low", min(prices))
#     high = kwargs.get("price_high", max(prices))
#     return [price - price * discount / 100 for price in prices if low <= price <= high]

# print(final_price(100, 200, 300, 400, 500, discount=5))
# print(final_price(100, 200, 300, 400, 500, discount=5, price_low=200))
# print(final_price(100, 200, 300, 400, 500, discount=5, price_high=200))
# print(final_price(100, 200, 300, 400, 500, discount=5, price_low=200, price_high=350))

Функция final_price была вызвана 1 раз(а)
[95.0, 190.0, 285.0, 380.0, 475.0]
Функция final_price была вызвана 2 раз(а)
[190.0, 285.0, 380.0, 475.0]
Функция final_price была вызвана 3 раз(а)
[95.0, 190.0]
Функция final_price была вызвана 4 раз(а)
[190.0, 285.0]


In [ ]:
@counter
# def fib(n):
#     if n < 2:
#         return n
#     return fib(n-2) + fib(n-1)

# fib(3)

Функция fib была вызвана 2 раз(а)
Функция fib была вызвана 4 раз(а)
Функция fib была вызвана 5 раз(а)
Функция fib была вызвана 5 раз(а)
Функция fib была вызвана 5 раз(а)


2

## Задание 4

In [ ]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}
  @wraps(func)
  def fmemo(*args):
      if args not in cache:
          cache[args] = func(*args)
      return cache[args]
  fmemo.cache = cache
  return fmemo

## Тестирование

In [ ]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 0.440753 секунд
Функция word_count вызвана с параметрами: ('whole'), {}
Функция word_count была вызвана 1 раз(а)
Cлово whole встречается 176 раз


In [ ]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [ ]:
#измеряем время

In [247]:
def fib(n):
 if n < 2:
  return n
 return fib(n-2) + fib(n-1)



n = 30
start = time.time()
fib(n)
end = time.time()

print("Время работы функции без с декоратора memo:", round(end - start, 10))

Время работы функции без с декоратора memo: 0.6442248821


In [248]:
start = time.time()
@memo
def fib(n):
 if n < 2:
  return n
 return fib(n-2) + fib(n-1)
end = time.time()
print("Время работы функции с декоратором memo:", round(end - start, 10))

Время работы функции с декоратором memo: 0.0002462864


**Вывод**: разница существенна: подсчет с использованием декоратора memo выполняется в разы быстрее